In [1]:
import numpy as np
import pandas as pd
import copy
import collections
from scipy import linalg
import math
from collections import defaultdict

In [2]:
#data
# index[0]:user_id, index[1]:movie_id, index[2]:ratings
df = pd.read_csv('./ml-100k/u.data',sep='\t',header=None)

In [3]:
df.head(3)

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


In [4]:
#movie list
df_info = pd.read_csv('./ml-100k/u.item',sep='|',header=None, encoding="ISO-8859-1")

In [5]:
df_info.head(2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
movielist = [df_info[1].tolist()[indx]+';'+str(indx+1) for indx in range(len(df_info[1].tolist()))]

In [7]:
nmovies = len(movielist)
nusers = len(df[0].drop_duplicates().tolist())
print("Movie counts: {}, User counts: {}".format(nmovies, nusers))

Movie counts: 1682, User counts: 943


In [8]:
min_ratings = 50
movies_rated  = list(df[1])
counts = collections.Counter(movies_rated)

In [9]:
dfout = pd.DataFrame(columns=['user']+movielist)

In [10]:
dfout.head(1)

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995);6,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,...,Mirage (1995);1673,Mamma Roma (1962);1674,"Sunchaser, The (1996);1675","War at Home, The (1996);1676",Sweet Nothing (1995);1677,Mat' i syn (1997);1678,B. Monkey (1998);1679,Sliding Doors (1998);1680,You So Crazy (1994);1681,Scream of Stone (Schrei aus Stein) (1991);1682


In [12]:
toremovelist = []
for i in range(1,nusers):
    tmpmovielist = [0 for j in range(nmovies)]
    dftmp =df[df[0]==i]
    for k in dftmp.index:
        if counts[dftmp.loc[k][1]]>= min_ratings:           
           tmpmovielist[dftmp.loc[k][1]-1] = dftmp.loc[k][2]
           
        else:
           toremovelist.append(dftmp.loc[k][1])
            
    dfout.loc[i] = [i]+tmpmovielist

In [13]:
toremovelist=list(set(toremovelist))
dfout.drop(dfout.columns[toremovelist], axis=1, inplace=True)
dfout.to_csv('utilitymatrix.csv',index=None)

In [14]:
dfout.head(5)

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,Richard III (1995);10,...,Cool Runnings (1993);1035,Hamlet (1996);1039,Forget Paris (1995);1041,Multiplicity (1996);1047,She's the One (1996);1048,Koyaanisqatsi (1983);1065,Shallow Grave (1994);1073,Reality Bites (1994);1074,Six Degrees of Separation (1993);1101,Some Kind of Wonderful (1987);1119
1,1,5,3,4,3,3,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,2,4,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,4,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 영화 장르 데이터

In [15]:
#matrix movies's content
movieslist = [int(m.split(';')[-1]) for m in dfout.columns[1:]]
moviescats = ['unknown','Action','Adventure','Animation','Children\'s','Comedy','Crime','Documentary',
              'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
              'Romance','Sci-Fi','Thriller','War','Western']
dfout_movies =  pd.DataFrame(columns=['movie_id']+moviescats)
startcatsindx = 5
cnt= 0
for m in movieslist:
    dfout_movies.loc[cnt] = [m]+df_info.iloc[m-1][startcatsindx:].tolist()
    cnt +=1 
print(dfout_movies.head())

dfout_movies.to_csv('movies_content.csv',index=None)

  movie_id unknown Action Adventure Animation Children's Comedy Crime  \
0        1       0      0         0         1          1      1     0   
1        2       0      1         1         0          0      0     0   
2        3       0      0         0         0          0      0     0   
3        4       0      1         0         0          0      1     0   
4        5       0      0         0         0          0      0     1   

  Documentary Drama Fantasy Film-Noir Horror Musical Mystery Romance Sci-Fi  \
0           0     0       0         0      0       0       0       0      0   
1           0     0       0         0      0       0       0       0      0   
2           0     0       0         0      0       0       0       0      0   
3           0     1       0         0      0       0       0       0      0   
4           0     1       0         0      0       0       0       0      0   

  Thriller War Western  
0        0   0       0  
1        1   0       0  
2        1 

### 결측치 대체(Imputation)

In [97]:
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in range(len(Ri)):
            Ri[i][Ri[i]==0] = sum(Ri[i])/float(len(Ri[i][Ri[i]>0]))
        return Ri
    def itemav():
        for i in range(len(Ri[0])):
            Ri[:,i][Ri[:,i]==0] = sum(Ri[:,i])/float(len(Ri[:,i][Ri[:,i]>0]))
        return Ri            
    switch = {'useraverage':userav(),'itemaverage':itemav()}
    return switch[inp]

### 유사도 평가

In [17]:
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def sim(x,y,metric='cos'):
    if metric == 'cos':
       return 1.-cosine(x,y)
    else:#correlation
       return pearsonr(x,y)[0]

## 협업 필터링 방법 (Collaborative Filtering Methods)

### 메모리 기반 협업 필터링

* 메모리 기반 형업 필터링은 **사용자 기반 협업 필터링(User-Based Collaborative Filtering)**과 **아이템 기반 협업 필터링(item-Based Collaborative Filtering)**으로 구분할 수 있다.

#### 1. 사용자 기반 협업 필터링

* **u_vec**은 사용자 평점 벡터로 함수 **FindKNeighbours**에서 이 벡터와 가장 비슷한 사용자 $K$를 찾는다. 
* **CalcRating** 함수는 앞의 식을 이용해서 예측 평점을 계산한다. 단, 분포 교정은 하지 않는다.
* 유틸리티 행렬이 데이터가 별로 없어 이웃이 없는 경우 사용자의 평균 평점으로 예측이 이루어지도록 구현되었다.
* 예측 평점이 5를 넘거나 1보다 낮은 경우 각각 5와 1로 조정된다.

In [18]:
def CF_userbased(u_vec,K,data,indxs=False):
    def FindKNeighbours(r,data,K):
        neighs = []
        cnt=0
        for u in range(len(data)):
            if data[u,r]>0 and cnt<K:
                neighs.append(data[u])   
                cnt +=1 
            elif cnt==K:
                break
        return np.array(neighs)
        
    def CalcRating(u_vec,r,neighs):
        rating = 0.
        den = 0.
        for j in range(len(neighs)):
            rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
            den += abs(neighs[j][-1])
        if den>0:
            rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:
            rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating>5:
            return 5.
        elif rating<1:
            return 1.
        return rating 
    #add similarity col
    data = data.astype(float)
    nrows = len(data)
    ncols = len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data
    #calc similarities:
    for u in range(nrows):
        if np.array_equal(data_sim[u,:-1],u_vec)==False: #list(data_sim[u,:-1]) != list(u_vec):
           data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
           data_sim[u,ncols] = 0.
    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    for r in range(ncols):
        if u_vec[r]==0:
           neighs = FindKNeighbours(r,data_sim,K)
           #calc the predicted rating
           u_rec[r] = CalcRating(u_vec,r,neighs)
    if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs] = -1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec    
    return u_rec

#### 2. 아이템 기반 협업 필터링

* 클래스 **CF_itembased**의 생성자는 아이템 유사도 행렬 **simmatrix**를 계산한다.
* 아이템에 대해 사용자 평점이 없는 경우 함수 **CalcRatings**를 통해 예측을 하게 되며, 이 때 함수 내부적으로 **simmatrix**를 사용한다.
* 함수 **GetKSimItemsperUser**는 사용자가 평가하지 않은 아이템과 유사도가 가장 높은 아이템 중 사용자가 과거에 평가했던 $K$개의 아이템을 찾는다.
* **u_vec**는 사용자 평점 벡터로, 유틸리티 행렬의 행 벡터이다. **CalcRatings**은 가중 평균 평점을 계산한다. 이웃을 찾을 수 없는 경우 평점은 아이템의 평균 평점으로 설정한다.

In [19]:
class CF_itembased(object):
    def __init__(self,data):
        #calc item similarities matrix
        nitems = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in range(nitems):
            for j in range(nitems):
                if j>=i:#triangular matrix
                    self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else:
                    self.simmatrix[i,j] = self.simmatrix[j,i]

    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
               neighitems.append(i)
               cnt+=1
            elif cnt==K:
               break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            rating +=  self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
        
    def CalcRatings(self,u_vec,K,indxs=False):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r]==0:
               neighitems = self.GetKSimItemsperUser(r,K,u_vec)
               #calc predicted rating
               u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs]=-1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        
            return recsvec
        return u_rec

### 트레이닝 셋과 테스트 셋 분리

In [20]:
def cross_validation(df,k):
    val_num = int(len(df)/float(k))
    print(val_num)
    df_trains = []
    df_vals = []
    for i in range(k):
        start_val = (k-i-1)*val_num
        end_val = start_val+val_num
        df_trains.append(pd.concat([df[:start_val],df[end_val:]]))
        df_vals.append(df[start_val:end_val])

    return df_trains,df_vals

### RMSE 평가

In [21]:
def SE(u_preds,u_vals):
    nratings = len(u_vals)
    se = 0.
    cnt = 0
    for i in range(nratings):
        if u_vals[i]>0:
           se +=  (u_vals[i]-u_preds[i])*(u_vals[i]-u_preds[i])
           cnt += 1
    return se,cnt

### 예측을 위해 U_VEC의 절반은 숨겨서 값을 예측하긴 위한 적업

In [22]:
import random
def HideRandomRatings(u_vec, ratiovals=0.5):
    u_test = np.zeros(len(u_vec))
    u_vals = np.zeros(len(u_vec))
    cnt = 0
    nratings = len(u_vec[u_vec>0])
    for i in range(len(u_vec)):
        if u_vec[i]>0:        
            if bool(random.getrandbits(1)) or cnt>=int(nratings*ratiovals):
                u_test[i]=u_vec[i]
            else:#random choice to hide the rating:
                cnt +=1
                u_vals[i]=u_vec[i]
    return u_test,u_vals

### 1. 데이터 로드

In [23]:
#load data
df = pd.read_csv('utilitymatrix.csv')
print(df.head(4))
df_movies = pd.read_csv('movies_content.csv')
movies = df_movies.values[:,1:]
print('check:::',len(df.columns[1:]),'--',len(df_movies))
movieslist = list(df.columns[1:])
#k-fold cv 5 folds
nfolds = 5
df_trains,df_vals = cross_validation(df,nfolds)

   user  Toy Story (1995);1  GoldenEye (1995);2  Four Rooms (1995);3  \
0     1                   5                   3                    4   
1     2                   4                   0                    0   
2     3                   0                   0                    0   
3     4                   0                   0                    0   

   Get Shorty (1995);4  Copycat (1995);5  Twelve Monkeys (1995);7  \
0                    3                 3                        4   
1                    0                 0                        0   
2                    0                 0                        0   
3                    0                 0                        0   

   Babe (1995);8  Dead Man Walking (1995);9  Richard III (1995);10  ...  \
0              1                          5                      3  ...   
1              0                          0                      2  ...   
2              0                          0                      0  

### 2. 데이터 준비

In [24]:
nmovies = len(df_vals[0].values[:,1:][0])
vals_vecs_folds = []
tests_vecs_folds = []
for i in range(nfolds):
    u_vecs = df_vals[i].values[:,1:]
    vtests = np.empty((0,nmovies),float)
    vvals = np.empty((0,nmovies),float)
    for u_vec in u_vecs:
        u_test,u_vals = HideRandomRatings(u_vec)
        vvals = np.vstack([vvals,u_vals])
        vtests = np.vstack([vtests,u_test])
    vals_vecs_folds.append(vvals)
    tests_vecs_folds.append(vtests)

### 3. 평가

* 아이템 기반 협업

In [25]:
err_itembased = 0.
cnt_itembased = 0

for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    cfitembased = CF_itembased(Umatrix)

    print('fold:',i+1)
    vec_vals = vals_vecs_folds[i]
    vec_tests = tests_vecs_folds[i]
    
    for j in range(len(vec_vals)):
        u_vals = vec_vals[j]
        u_test = vec_tests[j]
        
        #cf_itembased
        u_preds = cfitembased.CalcRatings(u_test,5)
        e,c =  SE(u_preds,u_vals)
        err_itembased +=e
        cnt_itembased +=c
        
rmse_itembased = np.sqrt(err_itembased/float(cnt_itembased))

print('user_itembased rmse:',rmse_itembased,'--',cnt_itembased)

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
user_itembased rmse: 1.031218673622769 -- 40024


In [26]:
u_preds

array([4., 3., 3., 4., 0., 4., 5., 4., 4., 5., 5., 4., 4., 3., 4., 4., 4.,
       3., 0., 4., 4., 4., 4., 4., 5., 3., 5., 4., 4., 3., 4., 3., 4., 4.,
       5., 4., 4., 4., 4., 4., 5., 4., 0., 4., 0., 5., 4., 4., 4., 4., 4.,
       0., 4., 0., 4., 4., 0., 4., 4., 4., 4., 0., 0., 0., 3., 4., 4., 5.,
       4., 4., 4., 0., 5., 4., 4., 5., 4., 3., 4., 0., 5., 4., 4., 0., 4.,
       4., 3., 4., 3., 3., 4., 4., 4., 4., 3., 3., 4., 3., 3., 4., 3., 4.,
       0., 3., 4., 5., 5., 4., 4., 4., 4., 4., 4., 3., 4., 3., 4., 0., 4.,
       4., 0., 4., 0., 4., 0., 4., 4., 4., 4., 4., 4., 4., 0., 4., 4., 0.,
       5., 5., 4., 4., 4., 5., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 0.,
       4., 5., 4., 4., 5., 0., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4.,
       0., 4., 4., 4., 4., 4., 0., 4., 4., 4., 4., 4., 0., 4., 5., 5., 5.,
       0., 4., 0., 4., 3., 5., 4., 4., 3., 0., 4., 4., 4., 4., 4., 4., 4.,
       0., 3., 4., 4., 4., 0., 4., 4., 2., 3., 3., 4., 3., 4., 4., 3., 4.,
       3., 3., 3., 3., 3.

In [27]:
u_vals

array([0., 0., 0., 0., 0., 5., 0., 0., 0., 5., 0., 4., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 3., 0., 0., 0., 0.,
       0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 4., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 3., 0., 0., 4., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 5., 5., 0., 4., 4., 0., 0., 5., 0.,
       0., 0., 0., 4., 0., 0., 0., 0., 0., 3., 0., 0., 3., 3., 0., 0., 0.,
       0., 0., 0., 2., 0., 4., 0., 0., 0., 0., 2., 4., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.,
       0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 3., 0., 0.

* 사용자 기반 협업

In [28]:
err_userbased = 0.
cnt_userbased = 0

for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    
    print('fold:',i+1)
    vec_vals = vals_vecs_folds[i]
    vec_tests = tests_vecs_folds[i]
    
    for j in range(len(vec_vals)):
        u_vals = vec_vals[j]
        u_test = vec_tests[j]
        
        #cf_userbased
        u_preds = CF_userbased(u_test,5,Umatrix)
        e,c =  SE(u_preds,u_vals)
        err_userbased +=e
        cnt_userbased +=c
        
rmse_userbased = np.sqrt(err_userbased/float(cnt_userbased))

print('user_userbased rmse:',rmse_userbased,'--',cnt_userbased)

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
user_userbased rmse: 1.0188729747700496 -- 40024


#### Classification Reports

In [32]:
def ClassificationMetrics(vec_vals,vec_recs,likethreshold=3,shortlist=50,ratingsval=False,vec_test=None):
    #convert vals in indxs vec
    indxs_like = [i for i in range(len(vec_vals)) if vec_vals[i]>likethreshold]
    indxs_dislike = [i for i in range(len(vec_vals)) if vec_vals[i]<=likethreshold and vec_vals[i]>0]
    cnt = len(indxs_like)+len(indxs_dislike)
    indxs_rec = []
    if ratingsval:
        #convert ratings into items's list
        #if vec_test==None:
        #    raise 'Error no test vector'
        indxs_rec = [i for i in range(len(vec_recs)) if vec_recs[i]>likethreshold and vec_test[i]<1][:shortlist]
    else:
        #consider only the first slot of recs
        indxs_rec = vec_recs[:shortlist]

    tp = len(set(indxs_rec).intersection(set(indxs_like)))
    fp = len(set(indxs_rec).intersection(set(indxs_dislike)))
    fn = len(set(indxs_like)^(set(indxs_rec).intersection(set(indxs_like))))
    precision = 0.
    if tp+fp>0:
        precision = float(tp)/(tp+fp)
    recall = 0.
    if tp+fn>0:
        recall = float(tp)/(tp+fn)
    f1 = 0.
    if recall+precision >0:
        f1 = 2.*precision*recall/(precision+recall)
    
    return np.array([precision,recall,f1]),cnt

#### 사용자 기반 CF의 Classification Report

In [33]:
u_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 4., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       4., 0., 0., 0., 0., 4., 0., 5., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 3., 0., 4., 0., 0., 0., 0., 0., 2., 0.,
       0., 1., 0., 3., 0., 3., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0.,
       5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [34]:
tot_measures = np.zeros(3)    
cnt_vals = 0.
#CF memory based
for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    
    print('fold:',i+1)
    tot_measures_fold = np.zeros(3)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    
    for j in range(len(vals_vecs)):
        u_vals = vals_vecs[j]
        u_test = tests_vecs[j]
        u_preds = CF_userbased(u_test,20,Umatrix)
        
        tmp_measures,cnt_tmp = ClassificationMetrics(u_vals,u_preds,3,50,True,u_test)
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
        
    tot_measures_fold /= float(len(vals_vecs))
    print(tot_measures_fold)
    tot_measures += tot_measures_fold
    
tot_measures /= float(nfolds)

print('/n')
print('precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals)

fold: 1
[0.58998995 0.16177406 0.24033692]
fold: 2
[0.5874215  0.18177075 0.26140035]
fold: 3
[0.62100324 0.19651447 0.27982898]
fold: 4
[0.60601139 0.186783   0.27172936]
fold: 5
[0.57911493 0.16985653 0.25197971]
/n
precision: 0.5967082000550411  recall: 0.179339759248636  f1: 0.2610550651696356 --- 40024.0


#### 아이템 기반 CF의 Classification Report

In [35]:
tot_measures = np.zeros(3)    
cnt_vals = 0.
#CF memory based
for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    cfitembased = CF_itembased(Umatrix)
    
    print('fold:',i+1)
    tot_measures_fold = np.zeros(3)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    
    for j in range(len(vals_vecs)):
        u_vals = vals_vecs[j]
        u_test = tests_vecs[j]
        u_preds = cfitembased.CalcRatings(u_test,5)
        
        tmp_measures,cnt_tmp = ClassificationMetrics(u_vals,u_preds,3,50,True,u_test)
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
        
    tot_measures_fold /= float(len(vals_vecs))
    print(tot_measures_fold)
    tot_measures += tot_measures_fold
    
tot_measures /= float(nfolds)

print('/n')
print('precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals)

fold: 1
[0.55105573 0.1254501  0.19363386]
fold: 2
[0.51647931 0.13648692 0.2023278 ]
fold: 3
[0.56409937 0.14854202 0.22298136]
fold: 4
[0.56506872 0.14570304 0.22286921]
fold: 5
[0.56351689 0.13064221 0.20506323]
/n
precision: 0.5520440024393244  recall: 0.1373648560085266  f1: 0.20937509185791675 --- 40024.0


### 모델 기반 협업 필터링 방법으로써, 교대 최소 제곱법(ALS, Alternating Least Square)와 확률 내리막 경사법(SGD, Stochatic Gradient Descent) 방법을 사용하여 추천시스템을 평가하시오.

In [95]:
from sklearn.decomposition import TruncatedSVD
def SVD(Umatrix,K,inp='none'):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    #imputation
    if inp != 'none':
        R_tmp = imputation(inp,Umatrix)     

    means = np.array([ R_tmp[i][R_tmp[i]>0].mean() for i in range(len(R_tmp))]).reshape(-1,1)
    R_tmp = R_tmp-means
    svd = TruncatedSVD(n_components=K, random_state=4)
    R_k = svd.fit_transform(R_tmp)
    R_tmp = svd.inverse_transform(R_k)
    R_tmp = means+R_tmp
    
    return np.round(R_tmp,0)

In [57]:
class CF_modelbased(object):
    def __init__(self,Movies,Umatrix,alpha=0.01,l=0.0001,its=50,tol=0.001):
        #calc parameters:
        self.nfeatures = len(Movies[0])+1#intercept
        nusers = len(Umatrix)
        nmovies = len(Umatrix[0])
        #add intercept col
        movies_feats = np.ones((nmovies,self.nfeatures))
        movies_feats[:,1:] = Movies
        self.movies_feats = movies_feats.astype(float)
        
        #set Umatrix as float
        self.Umatrix = Umatrix.astype(float)
        #initialize the matrix:
        Pmatrix = np.random.rand(nusers,self.nfeatures)
        Pmatrix[:,0]=1.
        err = 0.
        cost = -1
        for it in range(its):
            print ('it:',it,' -- ',cost)
            for u in range(nusers):
                for f in range(self.nfeatures):                    
                    if f==0:#no regularization
                        for m in range(nmovies):
                            if self.Umatrix[u,m]>0:
                                diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                                Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f])
                    else:
                        for m in range(nmovies):
                            if self.Umatrix[u,m]>0:
                                diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                                Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f] +l*Pmatrix[u][f])        
                
            cost = 0
            for u in range(nusers):
                for m in range(nmovies):
                    if self.Umatrix[u][m]>0:
                        cost += 0.5*pow(Umatrix[u][m]-np.dot(Pmatrix[u],self.movies_feats[m]),2)
                for f in range(1,self.nfeatures):
                    cost += float(l/2.0)*(pow(Pmatrix[u][f],2))
            if cost < tol:
                print('err',cost)
                break
        self.Pmatrix = Pmatrix
        
    def CalcRatings(self,u_vec):
        #find u_vec
        s = 0.
        u_feats = np.zeros(len(self.Pmatrix[0]))
        #in case the user is not present in the utility matrix find the most similar
        for u in range(len(self.Umatrix)):
            #print self.Umatrix[u]
            tmps = sim(self.Umatrix[u],u_vec)
            if tmps > s:
                s = tmps
                u_feats = self.Pmatrix[u]
            if s == 1.:
                break
        new_vec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r]==0:
                new_vec[r] = np.dot(u_feats,self.movies_feats[r])
        return new_vec

In [46]:
# ALS 함수

In [47]:
def ALS(Umatrix, K, iterations=50, l=0.001, tol=0.001):

    nrows = len(Umatrix)
    ncols = len(Umatrix[0])  
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    err = 0.
    Umatrix = Umatrix.astype(float)
    mask = Umatrix>0.
    mask[mask==True]=1
    mask[mask==False]=0
    mask = mask.astype(np.float64, copy=False)
    for it in range(iterations):
        for u, mask_u in enumerate(mask):
            P[u] = np.linalg.solve(np.dot(Qt, np.dot(np.diag(mask_u), Qt.T)) + l*np.eye(K), 
                                np.dot(Qt, np.dot(np.diag(mask_u), Umatrix[u].T))).T
        for i, mask_i in enumerate(mask.T):
            Qt[:,i] = np.linalg.solve(np.dot(P.T, np.dot(np.diag(mask_i), P)) + l*np.eye(K),
                                np.dot(P.T, np.dot(np.diag(mask_i), Umatrix[:,i])))                            
        err=np.sum((mask*(Umatrix - np.dot(P, Qt)))**2)
        if err < tol:
            break
    return np.round(np.dot(P,Qt),0)

In [38]:
# SGD 함수

In [39]:
def SGD(Umatrix, K, iterations=100, alpha=0.00001, l=0.001, tol=0.001):

    nrows = len(Umatrix)
    ncols = len(Umatrix[0])  
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    cost=-1
    for it in range(iterations):
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j] > 0:
                    eij = Umatrix[i][j] -np.dot(P[i,:],Qt[:,j])
                    for k in range(K):
                        P[i][k] += alpha*(2*eij*Qt[k][j]-l*P[i][k])
                        Qt[k][j] += alpha*(2*eij*P[i][k]-l*Qt[k][j]) 
        cost = 0
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j]>0:
                    cost += pow(Umatrix[i][j]-np.dot(P[i,:],Qt[:,j]),2)
                    for k in range(K):
                        cost += float(l/2.0)*(pow(P[i][k],2)+pow(Qt[k][j],2))
        if cost < tol:
            break
    return np.round(np.dot(P,Qt),0)

*ALS

In [58]:
err_als = 0.
cnt_als = 0

for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    
    print('fold:',i+1)
    teststartindx = len(Umatrix)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    
    for k in range(len(vals_vecs)):
        u_vals = vals_vecs[k]
        u_test = tests_vecs[k]
        
        #add test vector to utility matrix
        Umatrix = np.vstack([Umatrix,u_test])
    
    cbf_reg = CF_modelbased(movies,Umatrix)
    als_umatrix = ALS(Umatrix,20,50)
    
    #evaluate errors
    for indx in range(len(vals_vecs)):
        e,c = SE(als_umatrix[teststartindx+indx],vals_vecs[indx])
        err_als += e
        cnt_als += c
        u_preds = cbf_reg.CalcRatings(Umatrix[teststartindx+indx])


if cnt_als==0: cnt_als=1

rmse_als = np.sqrt(err_als/float(cnt_als))

print('als rmse:',rmse_als,'--',cnt_als)


fold: 1
it: 0  --  -1
it: 1  --  57484.523935691526
it: 2  --  47973.609303607125
it: 3  --  44143.0620519953
it: 4  --  41953.02164424427
it: 5  --  40474.39671169523
it: 6  --  39377.30126238336
it: 7  --  38514.09718528656
it: 8  --  37807.98557066014
it: 9  --  37214.49772539896
it: 10  --  36705.7163898031
it: 11  --  36262.97511286103
it: 12  --  35873.154222870355
it: 13  --  35526.6578690902
it: 14  --  35216.23937032606
it: 15  --  34936.28258277673
it: 16  --  34682.34200238323
it: 17  --  34450.836745101704
it: 18  --  34238.8399280421
it: 19  --  34043.92941826578
it: 20  --  33864.079369055646
it: 21  --  33697.57966148061
it: 22  --  33542.97493219333
it: 23  --  33399.01766208881
it: 24  --  33264.631561957205
it: 25  --  33138.88263243491
it: 26  --  33020.95603317092
it: 27  --  32910.13741051959
it: 28  --  32805.79768955081
it: 29  --  32707.38058769617
it: 30  --  32614.392287948467
it: 31  --  32526.39284106898
it: 32  --  32442.98896346891
it: 33  --  32363.827970

*SGB

In [64]:
err_sgd = 0.
cnt_sgd = 0

for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    print('fold:',i+1)
    teststartindx = len(Umatrix)
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    for k in range(len(vals_vecs)):
        u_vals = vals_vecs[k]
        u_test = tests_vecs[k]
        #add test vector to utility matrix
        Umatrix = np.vstack([Umatrix,u_test])
    
    cbf_reg = CF_modelbased(movies,Umatrix)
    sgd_umatrix = SGD(Umatrix,20,50)
    #evaluate errors
    for indx in range(len(vals_vecs)):
        e,c =  SE(sgd_umatrix[teststartindx+indx],vals_vecs[indx])
        err_sgd += e
        cnt_sgd += c
        u_preds = cbf_reg.CalcRatings(Umatrix[teststartindx+indx])


if cnt_sgd==0: cnt_sgd=1

rmse_sgd = np.sqrt(err_als/float(cnt_als))

print('sgd rmse:',rmse_sgd,'--',cnt_sgd)


fold: 1
it: 0  --  -1
it: 1  --  57326.63312955018
it: 2  --  47793.44816265422
it: 3  --  43970.152628409545
it: 4  --  41793.64492565604
it: 5  --  40328.27392405556
it: 6  --  39242.866783170444
it: 7  --  38389.78818162611
it: 8  --  37692.484348849976
it: 9  --  37106.729519053726
it: 10  --  36604.802321834
it: 11  --  36168.187245946974
it: 12  --  35783.88096296401
it: 13  --  35442.378743648
it: 14  --  35136.506696029544
it: 15  --  34860.70790132134
it: 16  --  34610.58579106236
it: 17  --  34382.600415481575
it: 18  --  34173.85946892324
it: 19  --  33981.970256771965
it: 20  --  33804.93215953052
it: 21  --  33641.05679745313
it: 22  --  33488.907632186725
it: 23  --  33347.25351735072
it: 24  --  33215.032460283335
it: 25  --  33091.322991066096
it: 26  --  32975.32128772674
it: 27  --  32866.322717560266
it: 28  --  32763.706808448016
it: 29  --  32666.92491390323
it: 30  --  32575.490014703217
it: 31  --  32488.968230553215
it: 32  --  32406.97171158799
it: 33  --  3232

아이템 기반 CF의 Classification Report

In [100]:
cnt_vals=0.
tot_measures = np.zeros(3)
for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    print('fold:',i+1)
    teststartindx = len(Umatrix)
    
    vals_vecs = vals_vecs_folds[i]
    tests_vecs = tests_vecs_folds[i]
    for k in range(len(vals_vecs)):
        u_vals = vals_vecs[k]
        u_test = tests_vecs[k]
        #add test vector to utility matrix
        Umatrix = np.vstack([Umatrix,u_test])
    
    svd_matrix = SVD(Umatrix,20,'itemaverage')
    
    tot_measures_fold = np.zeros(3)
    for indx in range(len(vals_vecs)):
        
        u_preds = svd_matrix[teststartindx+indx]
        
        tmp_measures,cnt_tmp = ClassificationMetrics(vals_vecs[indx],u_preds,3,50,False,Umatrix[teststartindx+indx])
        tot_measures_fold +=  tmp_measures
        cnt_vals += cnt_tmp
    tot_measures_fold = tot_measures_fold/float(len(vals_vecs))
    print(tot_measures_fold)
    tot_measures += tot_measures_fold
tot_measures = tot_measures/float(nfolds)
print('precision:',tot_measures[0],' recall:',tot_measures[1],' f1:',tot_measures[2],'---',cnt_vals)

fold: 1
[0.06648936 0.00170168 0.00330112]
fold: 2
[0.08244681 0.00257077 0.00493493]
fold: 3
[0.09308511 0.0020333  0.00394985]
fold: 4
[0.09663121 0.00336583 0.00634723]
fold: 5
[0.09308511 0.00329587 0.00608314]
precision: 0.08634751773049645  recall: 0.0025934915941447636  f1: 0.004923255360201149 --- 40024.0
